In [1]:
#!pip install --user plotly==4.1.0

In [2]:
import pandas as pd
import numpy as np
import qgrid

In [3]:
import plotly.graph_objects as go

In [4]:
periods = 4000
security_1 = np.cumsum(np.random.randn(periods)) + 100.
security_2 = security_1 + np.cumsum(np.random.randn(periods))*0.5 
dates = pd.date_range(start='06-01-2007', periods=periods, freq='d')
df = pd.DataFrame(index=dates, data={'security_1':security_1, 'security_2':security_2})


In [5]:
colors = np.random.randn(len(df))
sizes = np.random.rand(len(df))

layout = go.Layout(
    plot_bgcolor='Black',
    paper_bgcolor='Black',
    xaxis = {'showgrid': False, 'zeroline': False, 'showticklabels': False},
    yaxis = {'showgrid': False, 'zeroline': False, 'showticklabels': False},
    margin = {'l': 10, 'r': 10, 't': 10, 'b': 10},
)

scatter = go.Scatter(
    x=df['security_1'],
    y=df['security_2'],
    mode='markers',
    hoverinfo='none',
    marker={
        'line_width': 0,
        'color': colors,
        'size': sizes*15,
        'showscale': False,
        'colorscale': "Greens"
    }
)

fig = go.FigureWidget(
    data=[scatter],
    layout=layout,
 )

def lasso_event_handler(trace, points, state):
    inds = np.array(points.point_inds)
    qg.df = df.iloc[inds, :]

def lasso_unselect_handler(trade, points):
    qg.df = pd.DataFrame()
    
def hover_event_handler(trade, points, state):
    inds = np.array(points.point_inds)
    print(inds)

def click_event_handler(trade, points, state):
    inds = np.array(points.point_inds)
    print(inds)
    
#Note: Callbacks will only be triggered when the trace belongs to a
#    instance of plotly.graph_objs.FigureWidget and it is displayed in an
#    ipywidget context. Callbacks will not be triggered on figures
#   that are displayed using plot/iplot.

#contour = fig.add_histogram2dcontour(x=s1.x, y=s1.y, colorscale='Cividis',reversescale=False, hoverinfo='none', opacity=0.8)

# you need to use FigureWidget and then *not* use .show()
s1 = fig.data[0]     # <-- you need to create the figure first, then get scatter back to attach the event handlers
s1.on_selection(lasso_event_handler)
s1.on_hover(hover_event_handler)
s1.on_click(click_event_handler)
s1.on_deselect(lasso_unselect_handler)

In [6]:
df_selects = pd.DataFrame()
qg = qgrid.show_grid(df_selects)

In [7]:
# TODO: make the hover plots
# TODO: custom color scale
# TODO: node trace and edge trace

In [9]:
qg

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [10]:
def on_row_selected(change):
    print(f'New selected row index: {change.new[0]}')

qg.observe(on_row_selected, names=['_selected_rows'])

In [11]:
# Put everything together
fig

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'marker': {'color': array([ 0.49504889, -0.795…

In [12]:
# This will update the figure plot
fig.data[0].x = security_1

# Network Plot

In [13]:
import networkx as nx

In [14]:
# tested and works at 5000 nodes, 283k edges
G = nx.random_geometric_graph(250, 0.05)

In [15]:
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(   # go.Scattergl ( <-- lots of items need webgl
    x=edge_x, y=edge_y,
    line=dict(width=0.25, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.node[node]['pos']
    node_x.append(x)
    node_y.append(y)

    
# Hover and Click do not work on scattergl, but we need webgl bc plot is big (number of edges is large)
# Solution?: selector populates qgrid; trap row selection, and plot that
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=False,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2)
)

In [16]:
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency_iter()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

AttributeError: 'Graph' object has no attribute 'adjacency_iter'

In [17]:
layout=go.Layout(
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20,l=5,r=5,t=40),
    annotations=[ dict(
        text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
        showarrow=False,
        xref="paper", yref="paper",
        x=0.005, y=-0.002 ) ],
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

fig_n = go.FigureWidget(data=[edge_trace, node_trace], layout=layout)

fig_n

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': '#888', 'width': 0.25},
    …

In [19]:
len(edge_x)

720